In [ ]:
import pandas as pd
import os
import numpy as np

from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

# 1. Paths
csv_path     = '/content/drive/MyDrive/soil_classification/train_labels.csv'
image_folder = '/content/drive/MyDrive/soil_classification/train'

# 2. Load labels CSV
df = pd.read_csv(csv_path)
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(image_folder, x))

# 3. Encode soil_type → integer labels
soil_types = sorted(df['soil_type'].unique())   # ['alluvial','black','clay','red']
label_map  = {soil:i for i,soil in enumerate(soil_types)}
df['label'] = df['soil_type'].map(label_map)

# 4. Image preprocessing
IMG_H, IMG_W = 224, 224

def preprocess_image(path):
    img = load_img(path, target_size=(IMG_H, IMG_W))
    arr = img_to_array(img) / 255.0
    return arr

def build_dataset(df):
    X = np.stack([preprocess_image(p) for p in df['image_path']])
    y = df['label'].values
    return X, y

X, y = build_dataset(df)
y_cat = to_categorical(y, num_classes=len(soil_types))

# 5. Build model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_H, IMG_W, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(soil_types), activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6. Train
model.fit(X, y_cat,
          epochs=35,
          batch_size=32,
          validation_split=0.1,
          verbose=2)

# 7. Save model
model.save('/content/soil_model.h5')
print("✅ Model saved at /content/soil_model.h5")


started


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/35
35/35 - 40s - 1s/step - accuracy: 0.5942 - loss: 1.0009 - val_accuracy: 0.3984 - val_loss: 1.1081
Epoch 2/35
35/35 - 40s - 1s/step - accuracy: 0.7470 - loss: 0.6073 - val_accuracy: 0.7236 - val_loss: 0.9368
Epoch 3/35
35/35 - 39s - 1s/step - accuracy: 0.7989 - loss: 0.5137 - val_accuracy: 0.7642 - val_loss: 0.9567
Epoch 4/35
35/35 - 48s - 1s/step - accuracy: 0.8298 - loss: 0.4188 - val_accuracy: 0.2683 - val_loss: 1.7079
Epoch 5/35
35/35 - 75s - 2s/step - accuracy: 0.8717 - loss: 0.3555 - val_accuracy: 0.8780 - val_loss: 0.3810
Epoch 6/35
35/35 - 42s - 1s/step - accuracy: 0.8954 - loss: 0.3005 - val_accuracy: 0.7561 - val_loss: 1.0487
Epoch 7/35
35/35 - 37s - 1s/step - accuracy: 0.8981 - loss: 0.2939 - val_accuracy: 0.9106 - val_loss: 0.2522
Epoch 8/35
35/35 - 39s - 1s/step - accuracy: 0.9263 - loss: 0.1991 - val_accuracy: 0.9350 - val_loss: 0.1781
Epoch 9/35
35/35 - 37s - 1s/step - accuracy: 0.9390 - loss: 0.1767 - val_accuracy: 0.8537 - val_loss: 0.5819
Epoch 10/35
35/35 -

✅ Model saved at /content/soil_model.h5


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# --- Load the trained model ---
model = load_model('/content/soil_model.h5')
print("✅ Model loaded.")

# --- Constants ---
IMG_H, IMG_W = 128, 128
soil_types = ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']
label_map = {i: s for i, s in enumerate(soil_types)}

# --- Test images folder ---
test_folder = '/content/drive/MyDrive/soil_classification/test'

# --- Prediction function ---
def predict_soil_type(image_path):
    img = load_img(image_path, target_size=(IMG_H, IMG_W))
    arr = img_to_array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    pred = np.argmax(model.predict(arr, verbose=0), axis=1)[0]
    return label_map[pred]

# --- Predict and store results ---
results = []

for filename in sorted(os.listdir(test_folder)):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png','.gif','.webp')):
        path = os.path.join(test_folder, filename)
        pred = predict_soil_type(path)
        results.append({'image_id': filename, 'soil_type': pred})

# --- Save to CSV ---
output_df = pd.DataFrame(results)
csv_output_path = '/content/predictions.csv'
output_df.to_csv(csv_output_path, index=False)

print(f"✅ Predictions saved to {csv_output_path}")


✅ Model loaded.
✅ Predictions saved to /content/predictions.csv
